<a href="https://colab.research.google.com/github/Precious-Usanga/hamoye_stage_2_quiz/blob/master/hamoye_stageB_quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import pandas
import pandas as pd

#path of dataset
path = 'https://raw.githubusercontent.com/Precious-Usanga/hamoye_stage_2_quiz/master/energydata_complete.csv'
df = pd.read_csv(path)
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [2]:
df.drop(['lights', 'date'], axis=1, inplace=True)
df

In [28]:
# importing all other needed libraries
from pandas import Series
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing

In [7]:
# normalising the dataset to a common scale using the min max scaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
features_df = normalised_df.drop(columns=['Appliances'])
target_variable = normalised_df['Appliances']

In [8]:
# split features to represent training and testing dataset
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(features_df, target_variable, test_size=0.3, random_state=42)
print("number of test samples :", X_val.shape[0])
print("number of training samples:", X_train.shape[0])


number of test samples : 5921
number of training samples: 13814


In [9]:
#Define model
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()

#fit the model to the training dataset
linear_model.fit(X_train, y_train)

#obtain predictions
predicted_values = linear_model.predict(X_val)

# R^2 on the dataset
linear_model.score(X_val, y_val)

0.14890246319303535

In [10]:
# Question 12

linear_model.fit(df[['T2']],df[['T6']])
r2 = linear_model.score(df[['T2']],df[['T6']])
print('r2_score = ', r2.round(2))

r2_score =  0.64


In [11]:
# Question 13

from sklearn.metrics import mean_absolute_error
mse = mean_absolute_error(y_val, predicted_values)
print('The mean square error is: ', mse.round(2))

The mean square error is:  0.05


In [12]:
# Question 14

rss = np.sum(np.square(y_val - predicted_values))
print('THE RESIDUAL SUM OF SQUARE IS:',  round(rss, 2) )

THE RESIDUAL SUM OF SQUARE IS: 45.35


In [14]:
# Question 15
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_val, predicted_values))
print('THE ROOT MEAN SQUARE ERROR IS:',   round(rmse, 3))

THE ROOT MEAN SQUARE ERROR IS: 0.088


In [16]:
# Question 16
from sklearn.metrics import r2_score

r2_score = r2_score(y_val, predicted_values)
print('THE COEFFICIENT OF DETERMINATION IS:',   round(r2_score, 2) )

THE COEFFICIENT OF DETERMINATION IS: 0.15


In [35]:
# Question 17

def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

linear_model_weights = get_weights_df(linear_model, X_train, 'Linear_Model_Weight')
linear_model_weights

,Features,Linear_Model_Weight
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


In [25]:
# Question 18

from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4, random_state=2)
ridge_reg.fit(X_train, y_train)
rmse_ridge = np.sqrt(mean_squared_error(y_val, predicted_values))
print( 'THE NEW RMSE:', round(rmse_ridge, 3))

THE NEW RMSE: 0.088


In [38]:
# Question 19
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001, random_state=2)
lasso_reg.fit(X_train, y_train)

lasso_weights_df = get_weights_df(lasso_reg, X_train, 'Lasso_Weight')
lasso_weights_df
answer = lasso_weights_df.loc[lasso_weights_df['Lasso_Weight'] != 0]
answer
print("The number of non-zero feature weights is :", len(answer))

The number of non-zero feature weights is : 4


In [27]:
# Question 20

predicted_y_lasso = lasso_reg.predict(X_val)
rmse=np.sqrt(mean_squared_error(y_val, predicted_y_lasso ))
print('THE PREDICTED RMSE =', rmse.round(3))

THE PREDICTED RMSE = 0.094
